#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regression with TensorFlow

In a previous exercise, you worked with [Scikit Learn](https://scikit-learn.org/stable/) to define a linear regression model.   Recently you were introduced to [TensorFlow](https://www.tensorflow.org/), a powerful computational toolkit. We will now combine those learnings and create a linear regression model in TensorFlow.

## Overview

### Learning Objectives

  * Review the TensorFlow programming model
  * Use the `LinearRegressor` class in TensorFlow to predict median housing price, at the granularity of city blocks, based on one input feature
  * Evaluate the accuracy of a model's predictions using Root Mean Squared Error (RMSE)
  * Improve the accuracy of a model by tuning its hyperparameters

### Prerequisites

* Introduction to Colab
* Intermediate Python
* Introduction to Pandas
* Visualizations
* Introduction to TensorFlow

### Estimated Duration

60 minutes

### Grading Criteria

Each exercise is worth 3 points. The rubric for calculating those points is:

| Points | Description |
|--------|-------------|
| 0      | No attempt at exercise |
| 1      | Attempted exercise, but code does not run |
| 2      | Attempted exercise, code runs, but produces incorrect answer |
| 3      | Exercise completed successfully |

There are 3 exercises in this Colab so there are 9 points available. The grading scale will be 9 points.

## Problem Framing

Machine learning is not a solution looking for a problem, but is instead one of a variety of solutions that might work for an existing problem. Given this, we should begin our journey by understanding the problem we are trying to solve.

In this particular case, we would like to be able to **predict the price of a house in California**.

Questions we should ask ourselves might include:

*  Predict the price when? Now? In the past? In the future? For what range?
*  What is our tolerance for being wrong?
*  Are we okay with a few huge outliers if the overall model is better?
*  What metrics are we using to define success and what are the acceptable values?
*  Is there an non-ML way to solve this problem?
*  What data is available to solve the problem?

The list of questions is boundless. Eventually you'll need to move on, but understanding the problem and the solution space is vital.

---

For this problem we'll further define the problem by saying:

>  We want to create a system that predicts the prices of houses in California in 1990. We have census data from 1990 available to build and test the system. We will accept a system with a root mean squared error of 200,000 or better.

Since this is a contrived example we'll short-cut and say that our analysis has led us to believe that we want to use a linear regression model to serve as our prediction system.

## Data

The dataset we'll use for this Colab contains California housing data taken from the 1990 census data. This is a popular dataset for experimenting with machine learning models.

As with any data science project it is a good idea to take some time and review the [data schema and description](https://developers.google.com/machine-learning/crash-course/california-housing-data-description). Ask yourself:

* What data is available? What are the columns?
* What do those columns mean?
* What data types are those columns?
* What is the granularity of the data? In this particular case, what is a "block"?
* How many rows of data are there?
* Roughly how big is the data? Kilobytes? Megabytes? Gigabytes? Terabytes? More?

### Load the data

Now that we have a rough understanding of the data that we are going to use in our model, let's load it into this Colab and examine the data a little more closely.

We'll rely on Pandas to read a CSV version of the data from the internet.

In [0]:
import pandas as pd

housing_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

### Examine the data

You should always look at your data and statistics about that data before you begin modelling it. A great tool for getting a high-level view is to ask Pandas to describe the data.

In [0]:
housing_df.describe()

In this case we can see that all of the column counts are the same. That lets us know that every data point has a value. This can sometimes give you a false sense of security because many datasets have default values instead of empty values.

Looking at the min and max can be helpful too. Does a 1 value for a minimum number of rooms for a block match your mental model of what a block is?

As you probe a dataset you should ask yourself questions like this. When something doesn't look right, investigate it.

We can also identify the column of data that contains our target value. In this case we want to predict home values, so we will use `median_home_value` as our target.

Let's *imagine* that through more data analysis we decide that we'll use `total_rooms` as the feature that will be used to predict the home value.

It is also a good idea to take a look a the actual data. We can use Panda's `head` and `tail` methods to do this.

In [0]:
housing_df.head(20)

In [0]:
housing_df.tail(20)

Did you gain any insight from peeking at the actual data? Is the data sorted in a manner that might lead to a bad model?

In this case the data seems to be sorted ascending by longitude and possibly secondarily descending by latitude. We need to consider this when sampling or splitting the data.

### Prepare the data

A considerable amount of time is spent working with the dataset when creating a machine learning solution. In this case, we have looked at the data and it actually seems to be relatively clean.

The largest problem that we've seen is that there is an obvious sorting order to the data. To ensure that the sorting doesn't bite us later on, we should go ahead and randomize it now. A way to do this built into Pandas is to just create a 100% sample of the `DataFrame` in place of the original `DataFrame`.

The scale of the data across columns is also considerably different. It is often useful to normalize this data before feeding it to machine learning algorithms. We'll not do that now though since our intent for this lab is to build a simple linear regression model with one feature.

In [0]:
housing_df = housing_df.sample(frac=1)

housing_df.head()

### Train/Test Split

We want to go ahead and divide our data into testing and training splits. For this example we'll hold out 20% of the data for testing. Since the data is already shuffled, we can just take the first 20% and set it aside for testing and then take the final 80% and use it for training.

In [0]:
test_set_size = int(len(housing_df) * 0.2)

testing_df = housing_df[:test_set_size]
training_df = housing_df[test_set_size:]

print("Holding out {} records for testing. Using {} records for training.".format(len(testing_df), len(training_df)))

### Translating DataFrames to Datasets

`DataFrame` is a container for a dataset in Pandas. To process the data with TensorFlow we need to get the data in the `DataFrame` into a TensorFlow [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

Since our housing data fits in memory, we can use the `from_tensor_slices` class method to create our `Dataset`.

In [0]:
from tensorflow.data import Dataset

testing_ds = Dataset.from_tensor_slices(testing_df)
training_ds = Dataset.from_tensor_slices(training_df)

testing_ds, training_ds

The code above runs, but did it work? We can see that the shape is (9,) which tells us that the data sets have 9 columns and an unknown number of rows. The nine columns fits with our expectations, but it would be nice to know that our row counts are the same.

Intuitively you'd think this would be as simple as asking for the length of the data sets from Python:

```
 len(testing_ds)
 len(training_ds)
```

This won't work though. Remember that TensorFlow is just building a graph of things to run, but hasn't executed any of our graph yet. To do that we must create a session.

You also can't just ask for the count of rows in the dataset from the dataset itself. Why is this? The dataset doesn't necessarily know and it could be a very expensive operation.

The `Dataset` object can represent in-memory data, like what we have now. It can also represent data in multiple sources stored in different locations. In can even represent a stream of data that is never-ending.

Because of this we need to do a little more work to get a count of the data in a TensorFlow `Dataset`. To get a count we'll use the `reduce` operation. This operation takes an initial value, in our case 0, and then performs some function over and over for each row in the dataset. In this case we just add one for each value. The reduction returns values for each row and feeds it to the next. The final row simply returns the value to the runtime.

We can see below that the `reduce` operation counts the number of rows for the testing and training dataset and they both match the values we saw above in the Colab.

In [0]:
import numpy as np
import tensorflow as tf

session = tf.Session()

testing_ds_count = testing_ds.reduce(np.int64(0), lambda x, _: x + 1)
training_ds_count = training_ds.reduce(np.int64(0), lambda x, _: x + 1)

print(testing_ds_count)
print(training_ds_count)

print(session.run([testing_ds_count, training_ds_count]))

session.close()

## Build and Train the Model

In this section, we'll build a model to try to predict `median_house_value`, which will be our label (often called a target).  We'll use `total_rooms` as our input feature.

### LinearRegressor

To train our model, we'll use the [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) interface provided by the TensorFlow [Estimator](https://www.tensorflow.org/get_started/estimator) API. This API takes care of a lot of the low-level model plumbing, and exposes convenient methods for performing model training, evaluation, and inference.

Though the `LinearRegressor` has many configuration options, [only feature columns have to be specified when the regressor is created](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor#__init__).

We provide the regressor [feature columns](https://www.tensorflow.org/guide/feature_columns) as a list of columns that we'd like the model to use for training and prediction.

In [0]:
import tensorflow as tf

housing_features = [tf.feature_column.numeric_column("total_rooms")]

linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=housing_features,
)

### Input Function

The LinearRegressor that we just created is still not trained. To train the model we need to call the [train](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor#train) method and pass it an input function that feeds the regressor data.

The input function is responsible for creating the TensorFlow [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset). Let's look at a basic input function below.

In [0]:
import tensorflow as tf

from tensorflow.data import Dataset

def training_input():
  # First, we extract the features that we want to use to
  # train the model. In this case we are using the total_rooms
  # series from our housing data DataFrame.
  features = {
    'total_rooms': training_df['total_rooms'],
  }
  
  # Next we extract our labels (also called targets) from
  # the housing data DataFrame.
  labels = training_df['median_house_value']

  # We now create a TensorFlow Dataset object using the features
  # and labels.
  training_ds = Dataset.from_tensor_slices((features,labels))

  # We tell the Dataset to shuffle the order of the rows of data
  # passed to TensorFlow. We already shuffled the data once in
  # Pandas in order to create a training and testing set. We are
  # shuffling again because the data will be fed to TensorFlow
  # multiple times in batches. Shuffling adds some randomness
  # between batches.
  training_ds = training_ds.shuffle(buffer_size=10000)

  # We set the batch size. This will be the number of rows of
  # data that TensorFlow will operate on in each step of the
  # optimization.
  training_ds = training_ds.batch(100)

  # We now tell the Dataset to feed the entire training set five
  # times to the model.
  training_ds = training_ds.repeat(5)

  # And finally we return the Dataset to TensorFlow so that
  # the model can be trained.
  return training_ds

### Train

At this point training is as easy as calling the `train` method on the regressor and passing it the input function that we defined.

In [0]:
# Train the model
linear_regressor.train(
 input_fn=training_input,
)

We can see in the above output how TensorFlow's LinearRegressor will tell us, as it's training, what the loss is as the model improves. This output can be useful when, later on, we'll tweak the learning rate.

## Evaluate the Model

We have built and trained a `LinearRegressor`. Let's now use our regressor to make predictions about our test data and see how accurate it is.

### Input Function

We need a way to get the features that we'll be using for testing into our model for predictions. To do this we'll create an input function similar to the one above that we created for training.

You'll notice that the input function for prediction is much simpler than that for training. We simply need to create a `Dataset` containing the features that we'd like to use for prediction.

In [0]:
def testing_input():
  # Extract the features that we'd like to use for
  # prediction from our Pandas DataFrame.
  features = {
    'total_rooms': testing_df['total_rooms'],
  }

  # Create a TensorFlow Dataset of those features.
  testing_ds = Dataset.from_tensor_slices(features)

  # Set the batch size. The exact value isn't too
  # important here since we aren't training and only
  # need to send each row of data to TensorFlow once.
  # Batch size is a required setting, so we just set
  # it to one for this case.
  testing_ds = testing_ds.batch(1)

  return testing_ds

### Make Predictions

Now we need to make predictions using our test features. To do that we pass our testing input function to the `predict` method on our trained linear regressor.

In [0]:
predictions_node = linear_regressor.predict(
  input_fn=testing_input,
)

That runs pretty fast... almost suspiciously fast. The reason is that the model isn't actually making predictions at this point. We have just built the graph to make predictions. Since TensorFlow uses lazy execution the predictions won't be made until we ask for them.

Let's go ahead and get the predictions and put them in a NumPy array so that we can calculate our error.

In [0]:
predictions = np.array([item['predictions'][0] for item in predictions_node])
print("Our predictions: ", predictions)

### Evaluate Model

Now that we have predictions we can compare them to our actual values and evaluate the quality of our model.

In [0]:
import math

from sklearn import metrics

mean_squared_error = metrics.mean_squared_error(predictions, testing_df['median_house_value'])
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)

root_mean_squared_error = math.sqrt(mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

# Exercises

## Exercise 1

TensorFlow offers a variety of optimizers. We accepted the default in our example above. In this exercise we'll choose our own optimizer.

1. Check out the documentation for the [GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer).
1. Create an instance of `GradientDescentOptimizer` with a learning rate of 0.0000001 in the code block below.
1. Wrap the optimizer with a call to [tf.contrib.estimator.clip_gradients_by_norm](https://www.tensorflow.org/api_docs/python/tf/contrib/estimator/clip_gradients_by_norm) with a clip norm of 5.0.
1. Create a new `LinearRegressor`, passing it your new optimizer.

Is your root mean squared error better with this new optimizer?

### Student Solution

In [0]:
gd_optimizer = None # TODO: Create the optimizer here

#TODO: Clip the optimizer here

linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=housing_features,
    # TODO: Plug a new optmizer in right here
)

# Train the model
linear_regressor.train(
 input_fn=training_input,
)

# Make predictions
predictions_node = linear_regressor.predict(
  input_fn=testing_input,
)

# Convert the predctions to a NumPy array
predictions = np.array([item['predictions'][0] for item in predictions_node])

# Find the RMSE
root_mean_squared_error = math.sqrt(metrics.mean_squared_error(predictions, testing_df['median_house_value']))
root_mean_squared_error

### Answer Key

**Solution**

In [0]:
# TODO

**Validation**

In [0]:
# TODO

## Exercise 2

In this exercise we will build a model using a different feature. Choose a feature, say `housing_median_age` and use it in the place of `total_rooms`.

To do this you will need to:

1. Create a new training input function that uses the alternative feature
1. Create a new testing input function that use the alternative feature
1. Create a `LinearRegressor`
1. Train the model
1. Make predictions
1. Measure RMSE

### Student Solution

In [0]:
def training_input():
  # Your code goes here
  pass

def testing_input():
  # Your code goes here
  pass

housing_features = [] # TODO: Choose your housing features

linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=housing_features,
    # TODO: Use a custom optimizer and explore other hyperparameters if you would like 
)

# Train the model
linear_regressor.train(
 input_fn=training_input,
)

# Make predictions
predictions_node = linear_regressor.predict(
  input_fn=testing_input,
)

# Convert the predctions to a NumPy array
predictions = np.array([item['predictions'][0] for item in predictions_node])

# Find the RMSE
root_mean_squared_error = math.sqrt(metrics.mean_squared_error(predictions, testing_df['median_house_value']))
root_mean_squared_error

### Answer Key

**Solution**

In [0]:
# TODO

**Validation**

In [0]:
# TODO

## Exercise 3

In this exercise we will build a model using a multiple features. Choose a group of features and then:

1. Create a new training input function that uses the multiple features
1. Create a new testing input function that uses the multiple features
1. Create a `LinearRegressor`
1. Train the model
1. Make predictions
1. Measure RMSE

### Student Solution

In [0]:
def training_input():
  # TODO: Your code goes here
  pass

def testing_input():
  # TODO: Your code goes here
  pass

linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=housing_features,
    # TODO: Use a custom optimizer and explore other hyperparameters if you would like 
)

# Train the model
linear_regressor.train(
 input_fn=training_input,
)

# Make predictions
predictions = linear_regressor.predict(
  input_fn=testing_input,
)

# Convert the predctions to a NumPy array
predictions = np.array([item['predictions'][0] for item in predictions])

# Find the RMSE
root_mean_squared_error = math.sqrt(metrics.mean_squared_error(predictions, testing_df['median_house_value']))
root_mean_squared_error

### Answer Key

**Solution**

In [0]:
# TODO

**Validation**

In [0]:
# TODO

## Exercise 4: Challenge (Ungraded)

Given the [Kaggle Black Friday Sales](https://www.kaggle.com/mehdidag/black-friday) dataset use a TensorFlow `LinearRegressor` to predict the total price of the purchases for the day. This is the `sum(purchase)` for a shopper.

Features can include some combination of their **age**, **gender**, **occupation**, **city_category**, **stay_in_current_city_years**, and **marital_status**. Product and category data should not be used.

The data should be grouped by user for analysis.

Play with different optimizers, model settings, and data parameters (batch size, repeat, etc) to achieve the lowest RMSE that you can.

Work will likely include:

* Loading the data into Colab
* Examining the data quality
* Aggregating the data by user
* Examining the data
* Test/train split
* Building an input function for training
* Building an input function for testing
* Train model
* Make predictions
* Measure RSME
* Iterate

### Student Solution

In [0]:
# Your code goes here

### Answer Key

**Solution**

In [0]:
# TODO

**Validation**

In [0]:
# TODO